In [3]:
!pip install torch
!pip install opencv-python
!pip install umap
!pip install seaborn sentence-transformers textstat scikit-image fasttext



import pandas as pd
import numpy as np
import torch
import cv2
import umap
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from transformers import DistilBertTokenizer, DistilBertModel, pipeline
from sentence_transformers import SentenceTransformer
from textstat import flesch_reading_ease
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from skimage import io
from datasets import load_dataset
from scipy.stats import kurtosis  # Added for sharpness estimation
import fasttext.util
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor

# Download required NLTK data
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/priyanshisaraogi/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/priyanshisaraogi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/priyanshisaraogi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/priyanshisaraogi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/priyanshisaraogi/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [10]:
from datasets import load_dataset

category_reviews = "raw_review_All_Beauty"
category_meta = "raw_meta_All_Beauty"

# Load dataset from Hugging Face
dataset_reviews = load_dataset("McAuley-Lab/Amazon-Reviews-2023", category_reviews, split="full")
dataset_meta = load_dataset("McAuley-Lab/Amazon-Reviews-2023", category_meta, split="full")

# ✅ Step 1: Remove reviews with 0 helpful votes BEFORE sampling
dataset_reviews = dataset_reviews.filter(lambda x: x["helpful_vote"] > 0)


# Step 1: Remove reviews with 0 helpful votes
dataset_reviews = dataset_reviews.filter(lambda x: x["helpful_vote"] > 0)

# Step 2: Remove rows where 'text' is missing
dataset_reviews = dataset_reviews.filter(lambda x: x["text"] is not None and x["text"].strip() != "")

# Keep only reviews with at least one image
dataset_reviews = dataset_reviews.filter(lambda x: x.get("images") is not None and len(x["images"]) > 0)

# # ✅ Step 2: Sample 10% of filtered data
# num_samples = int(len(dataset_reviews) * 0.1)
# dataset_reviews_new = dataset_reviews.shuffle(seed=42).select(range(num_samples))

# num_samples_meta = int(len(dataset_meta) * 0.1)
# dataset_meta = dataset_meta.shuffle(seed=42).select(range(num_samples_meta))

# Convert to Pandas DataFrame
df_reviews = dataset_reviews.to_pandas()
df_meta = dataset_meta.to_pandas()

# Display the shape of the dataset
print(f"Filtered dataset size (before sampling): {len(dataset_reviews)} reviews")
print(f"Sampled dataset size (after selecting 10%): {df_reviews.shape[0]} reviews")
print(f"Sampled dataset size (after selecting 10%): {df_meta.shape[0]} reviews")

pd.set_option('display.max_colwidth', None)

# Display first few rows of the 'images' column
print(df_reviews['images'])

# Optionally save the final filtered dataset
df_reviews.to_csv("filtered_sampled_reviews.csv", index=False)
print("Final dataset saved as filtered_sampled_reviews.csv")


Filtered dataset size (before sampling): 26321 reviews
Sampled dataset size (after selecting 10%): 26321 reviews
Sampled dataset size (after selecting 10%): 112590 reviews
0                                                                                                                                                                                                                                                                                                                                     [{'attachment_type': 'IMAGE', 'large_image_url': 'https://images-na.ssl-images-amazon.com/images/I/41FMcmkZDLL._SL1600_.jpg', 'medium_image_url': 'https://images-na.ssl-images-amazon.com/images/I/41FMcmkZDLL._SL800_.jpg', 'small_image_url': 'https://images-na.ssl-images-amazon.com/images/I/41FMcmkZDLL._SL256_.jpg'}, {'attachment_type': 'IMAGE', 'large_image_url': 'https://images-na.ssl-images-amazon.com/images/I/41pcNrIv9YL._SL1600_.jpg', 'medium_image_url': 'https://images-na.ssl-images-amazon.co

In [ ]:
# Merge reviews with metadata on 'parent_asin'
df = pd.merge(df_reviews, df_meta, on='parent_asin', how='left')
print(f"Dataset size: {df.shape[0]} reviews and metadata")

Sampled dataset size (after selecting 10%): 26321 reviews


In [12]:
df['helpfulness_score'] = df['helpful_vote'] / (df['helpful_vote'].max() + 1)

In [13]:
# Initialize NLP Models
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
sia = SentimentIntensityAnalyzer()

In [14]:
# Text Preprocessing
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

In [15]:
# Apply text cleaning
df['cleaned_text'] = df['text'].astype(str).apply(clean_text)

In [16]:
from transformers import BertTokenizer, BertModel
import torch
# Load the pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Define a function to get BERT embeddings for a given text
def get_bert_embeddings(text):
    # Tokenize the input text and convert it to input IDs
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    
    # Get the BERT model's outputs
    with torch.no_grad():
        outputs = model(**inputs)
    
    # The output embeddings are in the last_hidden_state
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Take the mean of all token embeddings (for the entire sentence)
    
    return embeddings


In [ ]:
# # Get FastText Embeddings
# def get_fasttext_embedding(text):
#     words = text.split()
#     word_vectors = [ft.get_word_vector(word) for word in words]
#     if len(word_vectors) > 0:
#         return np.mean(word_vectors, axis=0)
#     else:
#         return np.zeros(300)

In [17]:
df['bert_embedding'] = df['cleaned_text'].apply(lambda x: get_bert_embeddings(x))

In [18]:
# Compute Other Text Features
df['sentiment'] = df['cleaned_text'].apply(lambda t: sia.polarity_scores(t)['compound'])
df['readability'] = df['cleaned_text'].apply(flesch_reading_ease)
df['review_length'] = df['cleaned_text'].apply(len)
df['punctuation_count'] = df['text'].apply(lambda t: t.count(".") if isinstance(t, str) else 0)

In [20]:
import cv2
import numpy as np
from scipy.ndimage import sobel

def estimate_image_quality(img):
    """ Compute an optimized quality score using Laplacian (blur), mean brightness, and Sobel (sharpness). """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Blur detection (Laplacian variance with slight Gaussian smoothing for noise reduction)
    blur_score = cv2.Laplacian(cv2.GaussianBlur(gray, (3,3), 0), cv2.CV_64F).var()

    # Brightness (Mean pixel value)
    brightness = np.mean(gray)

    # Faster Sharpness Estimation using Sobel Edge Magnitude
    sobel_x = sobel(gray, axis=0, mode='constant')
    sobel_y = sobel(gray, axis=1, mode='constant')
    sharpness = np.mean(np.sqrt(sobel_x**2 + sobel_y**2))

    # Weighted quality score (adjust weights if needed)
    quality_score = (blur_score * (-0.1)) + (sharpness * 0.7) + (brightness * 0.4)

    return quality_score


In [21]:
# Process images
def process_image(img_url):
    try:
        img = io.imread(img_url)
        return estimate_image_quality(img)
    except:
        return None  # Return None if the image cannot be processed

In [50]:
import numpy as np

def extract_large_image_urls(image_list):
    """
    Extract all large_image_urls from a list or numpy array of image dictionaries.
    Returns a list of URLs, or an empty list if none are found.
    """
    # Convert numpy.ndarray to list if necessary
    if isinstance(image_list, np.ndarray):
        image_list = image_list.tolist()
    
    # Now check if it's a list
    if not isinstance(image_list, list):
        print(f"Input is not a list: {image_list}")
        return []
    
    urls = []
    for img_dict in image_list:
        if not isinstance(img_dict, dict):
            print(f"Item is not a dict: {img_dict}")
            continue
        url = img_dict.get('large_image_url')
        if url:
            urls.append(url)
        else:
            print(f"No URL found in 'large_image_url'. Available keys: {list(img_dict.keys())}")
    
    return urls if urls else []

In [51]:
# Apply the function to extract URLs
df['image_urls'] = df['images_x'].apply(extract_large_image_urls)

# Print to verify
print(df['images_x'].head())
print(df['image_urls'].head())

sample = df['images_x'].iloc[4]
print("RAW:", sample)
print("EXTRACTED:", extract_large_image_urls(sample))


0                                                                                                                                                                                                                                                                                                                                 [{'attachment_type': 'IMAGE', 'large_image_url': 'https://images-na.ssl-images-amazon.com/images/I/41FMcmkZDLL._SL1600_.jpg', 'medium_image_url': 'https://images-na.ssl-images-amazon.com/images/I/41FMcmkZDLL._SL800_.jpg', 'small_image_url': 'https://images-na.ssl-images-amazon.com/images/I/41FMcmkZDLL._SL256_.jpg'}, {'attachment_type': 'IMAGE', 'large_image_url': 'https://images-na.ssl-images-amazon.com/images/I/41pcNrIv9YL._SL1600_.jpg', 'medium_image_url': 'https://images-na.ssl-images-amazon.com/images/I/41pcNrIv9YL._SL800_.jpg', 'small_image_url': 'https://images-na.ssl-images-amazon.com/images/I/41pcNrIv9YL._SL256_.jpg'}]
1    [{'attachment_type': 'IMAGE', 'large_i

In [52]:
def compute_average_quality(urls):
    if not urls:
        return None  # Skip if there are no URLs
    with ThreadPoolExecutor() as executor:
        scores = list(executor.map(process_image, urls))
    scores = [s for s in scores if s is not None]  # Remove None values
    return np.mean(scores) if scores else None  # Compute average score

In [53]:
from tqdm import tqdm

# Apply tqdm progress bar
tqdm.pandas(desc="Processing Images")

# Apply function with progress bar
df['avg_quality_score'] = df['image_urls'].progress_apply(compute_average_quality)


Processing Images: 100%|██████████| 26321/26321 [6:03:00<00:00,  1.21it/s]   


In [54]:
output_path = "processed_images.csv"
df.to_csv(output_path, index=False)
print(f"✅ DataFrame saved to: {output_path}")


✅ DataFrame saved to: processed_images.csv


In [ ]:
# Metadata Features
df['verified_purchase'] = df['verified_purchase'].astype(int)
print (df)

       rating  \
0         5.0   
1         4.0   
2         5.0   
3         5.0   
4         3.0   
...       ...   
26316     1.0   
26317     5.0   
26318     5.0   
26319     5.0   
26320     5.0   

                                                                                                   title_x  \
0                                                                           Doesn't Sting or Burn My Eyes!   
1                                                                                              Fun to Use!   
2                                                                                             Looks Great!   
3                                                          Big Bottle of Acetone-Based Nail Polish Remover   
4                  This fragrance is neither offensive nor enticing; it is very subtle & lasts a few hours   
...                                                                                                    ...   
26316  Huge red flags. We

In [56]:

# Install CLIP and image libraries
%pip install transformers torchvision pillow



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [61]:
# Install specific numpy version if needed
!pip install "numpy<2.0"

# Imports
from tqdm import tqdm
tqdm.pandas(desc="Processing Images")  # Register pandas extension with progress bar

from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
import torch
import numpy as np  # You missed this import

import transformers, PIL
print(transformers.__version__)  # Should be 4.44.2 or higher
print(torch.__version__)        # Should be 2.3.0 or higher
print(PIL.__version__)          # Should be 10.3.0 or higher
print(np.__version__)

# Load CLIP model and processor
try:
    clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
except Exception as e:
    print(f"Error loading model or processor: {e}")
    exit(1)

# Define image embedding function
def get_image_embedding(urls):
    try:
        if not urls:
            return np.zeros(512)
        image_url = urls[0]

        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(image_url, stream=True, headers=headers, timeout=10)
        response.raise_for_status()

        image = Image.open(response.raw).convert("RGB")
        image = image.resize((224, 224), Image.Resampling.LANCZOS)

        inputs = clip_processor(
            images=image,
            return_tensors="pt",
            do_resize=False,
            do_center_crop=False,
            do_normalize=True,
            padding=True
        )

        with torch.no_grad():
            emb = clip_model.get_image_features(**inputs)

        return emb.squeeze().numpy()

    except Exception as e:
        print(f"❌ Error for URL: {urls} — {e}")
        return np.zeros(512)


# ✅ Apply with progress bar
df['image_embedding'] = df['image_urls'].progress_apply(get_image_embedding)



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
4.51.2
2.2.2
11.1.0
1.26.4


Processing Images:   0%|          | 96/26321 [00:13<1:49:50,  3.98it/s]

❌ Error for URL: ['https://m.media-amazon.com/images/I/6196mT32ueL._SL1600_.jpg', 'https://m.media-amazon.com/images/I/61iXDoul3WL._SL1600_.jpg'] — 404 Client Error: Not Found for url: https://m.media-amazon.com/images/I/6196mT32ueL._SL1600_.jpg


Processing Images:   2%|▏         | 596/26321 [03:11<1:35:57,  4.47it/s]

❌ Error for URL: ['https://images-na.ssl-images-amazon.com/images/I/71I1z6tt1LL._SL1600_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71KZUgGwgdL._SL1600_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71dmUk8RH3L._SL1600_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71v9XgVYFBL._SL1600_.jpg'] — 404 Client Error: Not Found for url: https://images-na.ssl-images-amazon.com/images/I/71I1z6tt1LL._SL1600_.jpg


...

Processing Images:  98%|█████████▊| 25914/26321 [2:35:51<03:40,  1.85it/s]

❌ Error for URL: ['https://images-na.ssl-images-amazon.com/images/I/51kX8-pfRwL._SL1600_.jpg'] — 404 Client Error: Not Found for url: https://images-na.ssl-images-amazon.com/images/I/51kX8-pfRwL._SL1600_.jpg


Processing Images: 100%|█████████▉| 26288/26321 [2:38:54<00:13,  2.48it/s]

❌ Error for URL: ['https://images-na.ssl-images-amazon.com/images/I/71i50jeu4lL._SL1600_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71rxE6PpD5L._SL1600_.jpg'] — 404 Client Error: Not Found for url: https://images-na.ssl-images-amazon.com/images/I/71i50jeu4lL._SL1600_.jpg


Processing Images: 100%|██████████| 26321/26321 [2:39:12<00:00,  2.76it/s]


In [63]:
output_path = "processed_images.csv"
df.to_csv(output_path, index=False)
print(f"✅ DataFrame saved to: {output_path}")


✅ DataFrame saved to: processed_images.csv


In [64]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def chunk_text(text, chunk_size=128, stride=32):
    tokens = tokenizer(text, truncation=False, return_tensors="pt")["input_ids"][0]
    chunks = []
    for i in range(0, len(tokens), chunk_size - stride):
        chunk = tokens[i:i+chunk_size]
        chunks.append(chunk.tolist())
        if i + chunk_size >= len(tokens):
            break
    return chunks


In [65]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from datetime import datetime
from textstat import flesch_reading_ease
from nltk.sentiment import SentimentIntensityAnalyzer

# Show all columns in output
pd.set_option('display.max_columns', None)
print(df['image_embedding'][4])
print(df)

# Convert timestamp to datetime and calculate days since
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df['days_since_review'] = (datetime.now() - df['timestamp']).dt.days

# Calculate review length (number of words in 'text')
df['review_length'] = df['text'].apply(lambda x: len(str(x).split()))

# Normalize helpful votes (as proxy for helpfulness score)
df['helpfulness_score'] = df['helpful_vote'] / df['helpful_vote'].max()

# Calculate readability and sentiment score
df['readability'] = df['text'].apply(lambda x: flesch_reading_ease(str(x)))
sia = SentimentIntensityAnalyzer()
df['sentiment'] = df['text'].apply(lambda x: sia.polarity_scores(str(x))['compound'])


[ 6.85271770e-02  1.26189828e-01  2.83851773e-02  3.15660954e-01
  2.36579835e-01 -2.10076123e-02 -7.02859461e-02  1.37963489e-01
  3.53205204e-01  3.59326869e-01 -2.29127586e-01 -1.96678817e-01
 -8.12886655e-03  2.07439125e-01  2.62229025e-01  1.16304345e-01
  4.58405435e-01 -1.83797061e-01  7.47132301e-03 -4.77362752e-01
 -7.52998352e-01 -7.25376308e-02 -1.59005404e-01 -5.75190261e-02
 -3.04237902e-01  3.54136467e-01 -1.04230508e-01  3.47405195e-01
 -6.63421154e-02 -2.54648566e-01 -7.53595680e-02  6.41024947e-01
  5.41886449e-01 -3.35810184e-02  1.33811474e-01  4.63783592e-01
 -1.91866174e-01  6.77888691e-02 -5.84838867e-01  1.74281049e+00
 -3.08044076e-01 -4.90215778e-01  7.74513483e-02 -4.80359308e-02
  5.12056500e-02  9.85552967e-02 -7.06999153e-02  1.65635675e-01
 -9.27927941e-02  9.99877900e-02 -1.84481919e-01  1.94392934e-01
  3.09378564e-01  6.57349229e-02 -1.32552370e-01 -4.09587592e-01
  2.27522701e-02  2.60896772e-01 -5.48661500e-02  3.79980296e-01
 -1.10211745e-01  6.50613

In [66]:
output_path = "processed_images.csv"
df.to_csv(output_path, index=False)
print(f"✅ DataFrame saved to: {output_path}")


✅ DataFrame saved to: processed_images.csv


In [ ]:

# # Optional: If your metadata has image URLs
# if 'imageURLHighRes' in df_meta.columns:
#     df_meta['image_embedding'] = df_meta['imageURLHighRes'].apply(lambda x: get_image_embedding(x[0]) if isinstance(x, list) and len(x) > 0 else np.zeros(512))

# # Merge metadata with reviews
# df_merged = df_reviews.merge(df_meta, on='asin', how='left')

# # Chunk the review text
# df_merged['text_chunks'] = df_merged['review_body'].apply(chunk_text)


In [ ]:
# # CLIP-based image-text similarity (alignment) score
# def image_text_similarity(text, image):
#     inputs = clip_processor(text=[text], images=image, return_tensors="pt", padding=True)
#     outputs = clip_model(**inputs)
#     return torch.cosine_similarity(outputs.image_embeds, outputs.text_embeds).item()


In [ ]:
# # Reviewer-level features (if reviewerID is available)
# reviewer_stats = df.groupby("reviewerID").agg({
#     "reviewText": "count",
#     "helpful": lambda x: sum([h[0]/h[1] if h[1] != 0 else 0 for h in x])
# }).rename(columns={
#     "reviewText": "review_count_by_user",
#     "helpful": "avg_helpfulness_by_user"
# })

# df = df.merge(reviewer_stats, on="reviewerID", how="left")


In [ ]:
# # Add target labels
# def label_helpfulness(row):
#     if row['total_votes'] == 0:
#         return "no_votes"
#     ratio = row['helpful_votes'] / row['total_votes']
#     if ratio >= 0.75:
#         return "helpful"
#     elif ratio >= 0.5:
#         return "somewhat_helpful"
#     else:
#         return "not_helpful"

# df["helpfulness_label"] = df.apply(label_helpfulness, axis=1)
# df["helpfulness_regression"] = df["helpful_votes"] / df["total_votes"].replace(0, 1)


In [ ]:
# # Filter bad data
# df = df[df['total_votes'] > 0]
# df = df[df['reviewText'].str.len() > 5]
# df = df[df['imageURLHighRes'].notna() & df['imageURLHighRes'].str.len() > 0]
